In [16]:
import pandas as pd
import numpy as np
import os
import sklearn
import psycopg2
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

Connect to Postgresql Database

In [17]:
db_user = "postgres"
db_password = "Password021"
db_host = "localhost"
db_port = "5432"
db_name = "mlshopping"
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

tables_list = ["dim_month", "dim_revenue", "dim_visitor_type", "dim_weekend", "fact_online_shoppers", "online_shoppers_intention"]
df_dict = dict()
for table in tables_list:
    sql_query = f"Select * from {table}"
    df = pd.read_sql_query(sql_query, engine)
    print("✅ Data loaded successfully:", df.shape)
    print(df.head())
    df_dict[table] = df

print(df_dict)

✅ Data loaded successfully: (12, 2)
   month_id month_name
0         1        jan
1         2        feb
2         3        mar
3         4        apr
4         5        may
✅ Data loaded successfully: (2, 2)
   revenue_id  revenue
0           1    False
1           2     True
✅ Data loaded successfully: (3, 2)
   visitor_type_id       visitor_type
0                1        new_visitor
1                2              other
2                3  returning_visitor
✅ Data loaded successfully: (2, 2)
   weekend_id  weekend
0           1    False
1           2     True
✅ Data loaded successfully: (73980, 19)
   fact_id  administrative  administrative_duration  informational  \
0        1             4.0               659.506667            3.0   
1        2             0.0                 0.000000            0.0   
2        3             0.0                 0.000000            0.0   
3        4             9.0               135.100000            0.0   
4        5             0.0               

Bool to int functions and Model training, splitting Train and Test data , ML (RandomForest)

In [18]:
df = df_dict["fact_online_shoppers"]

df['revenue_id'] = df['revenue_id'].astype(int)

categorical_cols = df.select_dtypes(include=['object']).columns
encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])


df = df.fillna(0)

# -------------------------------
# 🔹 تقسیم داده‌ها
# -------------------------------
X = df.drop('revenue_id', axis=1)
y = df['revenue_id']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)


print("\n✅ Model Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))



✅ Model Accuracy: 1.0

📊 Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00     12598
           2       1.00      1.00      1.00      2198

    accuracy                           1.00     14796
   macro avg       1.00      1.00      1.00     14796
weighted avg       1.00      1.00      1.00     14796

